In [1]:
import pandas as pd
import re
import numpy as np

# Data Prep
Preparing dataset for Bayes net.

<ul>
    <li>Group according to NJ clustering.</li>
    <li>Remove uninformative variables, with less than 5% positive response.</li>
</ul>

In [2]:
op_cohort = pd.read_csv('../data/op_cohort.csv')
X_clusters = pd.read_csv('../results/clustering/nj_clustering.csv')

In [3]:
# Group according to NJ clustering

for i in range(1,16):
    painful_areas = X_clusters.loc[X_clusters['cluster'] == i, 'painful_area'].values.tolist()
    cluster_label = 'cluster_%d' % i
    op_cohort[cluster_label] = op_cohort[painful_areas].sum(axis=1)
    op_cohort.loc[op_cohort[cluster_label] > 0, cluster_label] = 1
    
painful_area_columns = [col for col in op_cohort.columns if re.search(r'^[F|B|P][0-9]', col)]
op_cohort = op_cohort.drop(columns=painful_area_columns)

In [4]:
# Remove uninformative variables

threshold = int(np.floor(len(op_cohort.index) * 0.05))

positive_responses = op_cohort.drop(columns=['final_diagnosis_DV']) > 0
positive_responses = positive_responses.sum()
columns_to_remove = positive_responses.loc[positive_responses < threshold].index
op_cohort.drop(columns=columns_to_remove, inplace=True)
op_cohort = op_cohort.astype(int)

In [5]:
op_cohort.to_csv('../data/curated_pain_data.csv', index=False)